
HELLO ALL, MY APPROACH IS AS FOLLOWS: PRE PROCESS DATA, THEN BUILD MODELS TO PREDICT FIRST - QV2M, SECOND - T2M AND THIRD THE VACATION_RATE.
<BR>
AFTER TRAINING THE MODELS:
UPLOAD TEST DATASET
PREDICT QV2M AND APPEND TO DATASET
THIS WILL BE INPUT FOR T2M PREDICTION
SIMILARLY THE OUTPUT OF T2M PREDICTION IS APPENDED TO THE DATASET
WHICH ACTS LIKE INPUT FOR VACATION_RATE PREDICTOR
FINAL PREDICTION IS AGAIN APPENDED.

In [34]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [35]:
df = pd.read_csv('train.csv')
df

,ID,YEAR,T2M_RANGE,T2M_MAX,T2M_MIN,RH2M,PS,WS10M,QV2M,T2M,VACATION_RATE
0,0,1982.0,-241.552430,29.798545,13.691128,71.652934,89.725069,3.658959,8.474319,21.723403,64.184741
1,1,1982.0,-241.819997,30.902708,15.298046,69.754933,89.865668,3.529113,8.988374,22.911522,42.649137
2,2,1982.0,-241.883202,30.296561,14.660419,66.661416,89.846273,4.217007,8.474923,22.994887,52.819141
3,3,1982.0,-242.007629,30.803224,15.246491,67.621173,90.029795,4.484080,9.005508,23.594655,39.445489
4,4,1982.0,-240.659332,32.500906,15.303059,62.097329,89.825707,3.057211,8.307515,24.536776,46.444289
...,...,...,...,...,...,...,...,...,...,...,...
1434,1434,2009.0,-246.654914,30.644813,20.268662,78.611410,89.333566,6.433604,13.258977,26.685352,6.391643
1435,1435,2009.0,-247.651283,29.857874,20.425241,78.136636,89.324790,7.638996,12.717079,25.991700,1.920453
1436,1436,2009.0,-247.594571,30.330303,21.123513,77.497787,89.334675,8.939772,13.058350,26.619025,7.144639
1437,1437,2009.0,-247.083708,30.646652,20.883603,75.857655,89.465743,7.663743,12.805387,26.923294,4.886901


In [36]:
df = df[df['VACATION_RATE'] > 0]
df = df[df['VACATION_RATE'] <= 100]

In [37]:
df.describe()

,ID,YEAR,T2M_RANGE,T2M_MAX,T2M_MIN,RH2M,PS,WS10M,QV2M,T2M,VACATION_RATE
count,1388.000000,1388.000000,1388.000000,1388.000000,1388.000000,1388.000000,1388.000000,1388.000000,1388.000000,1388.000000,1388.000000
mean,722.092939,1995.350865,-244.380572,32.528835,19.648124,68.779115,89.563738,4.177282,11.332048,27.277948,20.180588
std,416.044961,7.980516,3.335121,4.013461,2.816027,15.907952,0.246157,1.472831,2.390980,3.350290,21.929395
min,0.000000,1982.000000,-250.797410,25.301324,11.065917,25.644879,88.908008,1.565365,4.885502,19.223138,0.004467
25%,359.750000,1988.000000,-247.126756,29.386977,17.915525,57.128561,89.373697,3.133437,9.279800,25.200899,4.116285
50%,723.500000,1995.000000,-245.158344,31.315237,20.295016,74.368213,89.543157,3.832753,12.202225,26.615474,11.585173
75%,1081.250000,2002.000000,-241.659481,35.419248,21.418174,80.735249,89.747831,4.845757,13.280411,29.119557,29.649595
max,1438.000000,2009.000000,-235.912719,42.524543,26.114388,93.728038,90.267493,9.222617,15.113460,36.215568,99.507463


In [38]:
corr = df.corr()
corr.style.background_gradient(cmap='inferno')

,ID,YEAR,T2M_RANGE,T2M_MAX,T2M_MIN,RH2M,PS,WS10M,QV2M,T2M,VACATION_RATE
ID,1.000000,0.999365,-0.068701,-0.079745,-0.017075,0.108280,0.018307,-0.037086,0.101989,-0.057523,0.094241
YEAR,0.999365,1.000000,-0.047507,-0.056999,-0.013757,0.084397,0.018433,-0.038491,0.084023,-0.040550,0.105040
T2M_RANGE,-0.068701,-0.047507,1.000000,0.738471,-0.312970,-0.883379,0.405394,-0.272386,-0.934248,0.344876,0.499220
T2M_MAX,-0.079745,-0.056999,0.738471,1.000000,0.407831,-0.932830,-0.072268,-0.180741,-0.555792,0.884054,0.143520
T2M_MIN,-0.017075,-0.013757,-0.312970,0.407831,1.000000,-0.117202,-0.649992,0.114633,0.483297,0.778744,-0.475324
RH2M,0.108280,0.084397,-0.883379,-0.932830,-0.117202,1.000000,-0.139141,0.231525,0.800579,-0.700235,-0.305820
PS,0.018307,0.018433,0.405394,-0.072268,-0.649992,-0.139141,1.000000,-0.383566,-0.515070,-0.369413,0.285471
WS10M,-0.037086,-0.038491,-0.272386,-0.180741,0.114633,0.231525,-0.383566,1.000000,0.252845,-0.099928,-0.191108
QV2M,0.101989,0.084023,-0.934248,-0.555792,0.483297,0.800579,-0.515070,0.252845,1.000000,-0.142264,-0.498291
T2M,-0.057523,-0.040550,0.344876,0.884054,0.778744,-0.700235,-0.369413,-0.099928,-0.142264,1.000000,-0.145777


# 1 PRED QV2M

In [39]:
df1 = df.drop(['VACATION_RATE', 'T2M'], axis=1)
df1

,ID,YEAR,T2M_RANGE,T2M_MAX,T2M_MIN,RH2M,PS,WS10M,QV2M
0,0,1982.0,-241.552430,29.798545,13.691128,71.652934,89.725069,3.658959,8.474319
1,1,1982.0,-241.819997,30.902708,15.298046,69.754933,89.865668,3.529113,8.988374
2,2,1982.0,-241.883202,30.296561,14.660419,66.661416,89.846273,4.217007,8.474923
3,3,1982.0,-242.007629,30.803224,15.246491,67.621173,90.029795,4.484080,9.005508
4,4,1982.0,-240.659332,32.500906,15.303059,62.097329,89.825707,3.057211,8.307515
...,...,...,...,...,...,...,...,...,...
1434,1434,2009.0,-246.654914,30.644813,20.268662,78.611410,89.333566,6.433604,13.258977
1435,1435,2009.0,-247.651283,29.857874,20.425241,78.136636,89.324790,7.638996,12.717079
1436,1436,2009.0,-247.594571,30.330303,21.123513,77.497787,89.334675,8.939772,13.058350
1437,1437,2009.0,-247.083708,30.646652,20.883603,75.857655,89.465743,7.663743,12.805387


In [40]:
X_1 = df1.iloc[:, :-1].values
y_qv = df1.iloc[:, -1].values

In [41]:
from sklearn.model_selection import train_test_split
X_qv_train, X_qv_val, y_qv_train, y_qv_val = train_test_split(X_1, y_qv, test_size = 0.2, random_state = 42)

In [42]:
from sklearn.linear_model import LinearRegression
regressor_qv = LinearRegression()
regressor_qv.fit(X_qv_train, y_qv_train)
np.set_printoptions(precision=2)

In [43]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_qv_pred = regressor_qv.predict(X_qv_val)
qv_rmse = mean_squared_error(y_qv_val, y_qv_pred, squared=False)
print("QV2M RMSE:", qv_rmse)

QV2M RMSE: 0.26517088741476136


In [44]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = regressor_qv, X = X_qv_train, y = y_qv_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 98.65 %
Standard Deviation: 0.12 %


# 2 PRED T2M

In [45]:
df2 = df.drop(['VACATION_RATE'], axis=1)
df2

,ID,YEAR,T2M_RANGE,T2M_MAX,T2M_MIN,RH2M,PS,WS10M,QV2M,T2M
0,0,1982.0,-241.552430,29.798545,13.691128,71.652934,89.725069,3.658959,8.474319,21.723403
1,1,1982.0,-241.819997,30.902708,15.298046,69.754933,89.865668,3.529113,8.988374,22.911522
2,2,1982.0,-241.883202,30.296561,14.660419,66.661416,89.846273,4.217007,8.474923,22.994887
3,3,1982.0,-242.007629,30.803224,15.246491,67.621173,90.029795,4.484080,9.005508,23.594655
4,4,1982.0,-240.659332,32.500906,15.303059,62.097329,89.825707,3.057211,8.307515,24.536776
...,...,...,...,...,...,...,...,...,...,...
1434,1434,2009.0,-246.654914,30.644813,20.268662,78.611410,89.333566,6.433604,13.258977,26.685352
1435,1435,2009.0,-247.651283,29.857874,20.425241,78.136636,89.324790,7.638996,12.717079,25.991700
1436,1436,2009.0,-247.594571,30.330303,21.123513,77.497787,89.334675,8.939772,13.058350,26.619025
1437,1437,2009.0,-247.083708,30.646652,20.883603,75.857655,89.465743,7.663743,12.805387,26.923294


In [46]:
X_2 = df2.iloc[:, :-1].values
y_t = df2.iloc[:, -1].values

In [47]:
from sklearn.model_selection import train_test_split
X_t_train, X_t_val, y_t_train, y_t_val = train_test_split(X_2, y_t, test_size = 0.2, random_state = 0)

In [48]:
from sklearn.linear_model import LinearRegression
regressor_t = LinearRegression()
regressor_t.fit(X_t_train, y_t_train)
np.set_printoptions(precision=2)

In [49]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_t_pred = regressor_t.predict(X_t_val)
t_rmse = mean_squared_error(y_t_val, y_t_pred, squared=False)
print("T2M RMSE:", t_rmse)

T2M RMSE: 0.2544915670933947


In [50]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = regressor_t, X = X_t_train, y = y_t_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 99.41 %
Standard Deviation: 0.10 %


# 3 A NOT CONSIDERED

In [51]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers

In [52]:
# df3 = df
# df3

In [53]:
# X_3 = df3.drop(['VACATION_RATE'], axis=1)
# y_vr = df['VACATION_RATE']
# X_vr_train, X_vr_val, y_vr_train, y_vr_val = train_test_split(
#     X_3, y_vr, test_size=0.2, random_state=42)

In [54]:
# scaler = StandardScaler()
# X_vr_train_scaled = scaler.fit_transform(X_vr_train)
# X_vr_val_scaled = scaler.transform(X_vr_val)

In [55]:
# model = keras.Sequential([
#     layers.Input(shape=(X_vr_train_scaled.shape[1],)),
#     layers.Dense(128, activation='relu'),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(32, activation='relu'),
#     layers.Dense(1)
# ])

# model.compile(optimizer='adam', loss='mean_squared_error')

In [56]:
# model.fit(X_vr_train_scaled, y_vr_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)


In [57]:
# y_vr_pred = model.predict(X_vr_val_scaled)


In [58]:
# vacation_rmse = mean_squared_error(y_vr_val, y_vr_pred, squared=False)
# print("VACATION_RATE RMSE:", vacation_rmse)

## 3 B NOT CONSIDERED


In [59]:
# from sklearn.ensemble import GradientBoostingRegressor

In [60]:
# model_vacation_rate = GradientBoostingRegressor(n_estimators=100, random_state=42)
# model_vacation_rate.fit(X_vr_train, y_vr_train)

In [61]:
# y_vr_pred_1 = model_vacation_rate.predict(X_vr_val)

In [62]:
# rmse_vacation_rate =  mean_squared_error(y_vr_val, y_vr_pred_1, squared=False)
# print("VACATION_RATE RMSE:", rmse_vacation_rate)

# 3 C TO BE CONSIDERED FOR PRED OF VAC_RATE

In [63]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt



In [64]:
df_XGB = df
X_3_XGB = df_XGB.iloc[:, :-1].values
y_vr_XGB = df_XGB.iloc[:, -1].values

# X_vr_XGB = df_XGB  # Features: Predicted T2M, Predicted QV2M, Relative Humidity, Surface Pressure
# y_vr_XGB = df_XGB['VACATION_RATE']
X_vr_train_XGB, X_vr_val_XGB, Y_vr_train_XGB, Y_vr_val_XGB = train_test_split(X_3_XGB, y_vr_XGB, test_size=0.2, random_state=42)

In [65]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 2, 3],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'objective': ['reg:squarederror']
}

xgb_model = XGBRegressor(random_state=42)

In [66]:
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1  )

In [67]:
random_search.fit(X_vr_train_XGB, Y_vr_train_XGB)
best_xgb_model = random_search.best_estimator_
Y_pred_XGB = best_xgb_model.predict(X_vr_val_XGB)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


In [68]:
rmse_vacation_rate = sqrt(mean_squared_error(Y_vr_val_XGB, Y_pred_XGB))

# print(random_search.best_params_)
print(f"RMSE for VACATION_RATE predictions (XGBoost): {rmse_vacation_rate:.2f}")

RMSE for VACATION_RATE predictions (XGBoost): 3.10


# 4

In [69]:
df_test_1 = pd.read_csv('test.csv')
df_test_1

,ID,YEAR,T2M_RANGE,T2M_MAX,T2M_MIN,RH2M,PS,WS10M
0,1440,2009.0,-309.569706,32.919144,17.735611,74.415249,82.443918,6.513046
1,1441,2009.0,-310.763533,32.340370,17.955298,75.232482,82.320169,5.163099
2,1442,2009.0,-313.096292,29.670217,17.788919,80.950265,82.456590,3.974139
3,1443,2009.0,-312.110915,29.586632,16.777071,82.254238,82.460926,5.504794
4,1444,2009.0,-312.445008,30.088295,17.233512,81.311915,82.326908,5.634695
...,...,...,...,...,...,...,...,...
695,2135,2022.0,-310.857597,28.357413,15.023100,86.599317,82.511107,2.637025
696,2136,2022.0,-312.033878,26.943856,14.236847,85.965911,82.748450,3.636588
697,2137,2022.0,-312.869384,26.542410,14.972372,88.082615,82.626626,3.840101
698,2138,2022.0,-308.938303,27.599339,12.882619,84.585012,82.632252,3.306362


In [70]:
y_qv_pred = regressor_qv.predict(df_test_1)
y_qv_pred
# df_test_1

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


array([ 15.38,  15.59,  15.99,  15.74,  15.9 ,  15.4 ,  16.91,  16.69,
        16.74,  15.97,  15.35,  13.62,  13.97,  15.29,  16.38,  16.23,
        15.25,  14.06,  12.14,  14.52,  13.52,  13.86,  12.08,  14.85,
        11.79,  11.52,  10.17,  10.52,  10.73,  10.56,   8.72,  10.55,
         9.93,   8.86,  12.08,  11.92,  13.46,  14.99,  14.8 ,  14.67,
        15.38,  16.54,  16.22,  15.98,  15.96,  16.18,  15.75,  15.39,
        15.21,  15.88,  15.61,  15.67,  15.73,  15.9 ,  16.3 ,  16.64,
        15.95,  15.95,  15.92,  16.28,  16.51,  16.87,  16.59,  16.46,
        16.64,  16.51,  16.53,  16.3 ,  15.6 ,  15.5 ,  14.86,  15.13,
        12.54,  13.08,  14.31,  11.15,  11.6 ,  11.81,  10.23,   8.53,
        10.27,  13.84,  10.15,   7.7 ,   9.88,  10.11,  10.44,  10.48,
        11.91,  13.35,  15.32,  14.97,  14.25,  15.15,  15.32,  15.99,
        15.06,  14.52,  14.82,  15.31,  14.4 ,  14.46,  14.97,  15.38,
        15.09,  15.4 ,  15.86,  15.65,  15.4 ,  15.67,  15.6 ,  15.22,
      

In [71]:
df_test_2 = df_test_1
df_test_2["QV2M"] = y_qv_pred
df_test_2

,ID,YEAR,T2M_RANGE,T2M_MAX,T2M_MIN,RH2M,PS,WS10M,QV2M
0,1440,2009.0,-309.569706,32.919144,17.735611,74.415249,82.443918,6.513046,15.378905
1,1441,2009.0,-310.763533,32.340370,17.955298,75.232482,82.320169,5.163099,15.590121
2,1442,2009.0,-313.096292,29.670217,17.788919,80.950265,82.456590,3.974139,15.986008
3,1443,2009.0,-312.110915,29.586632,16.777071,82.254238,82.460926,5.504794,15.744121
4,1444,2009.0,-312.445008,30.088295,17.233512,81.311915,82.326908,5.634695,15.898722
...,...,...,...,...,...,...,...,...,...
695,2135,2022.0,-310.857597,28.357413,15.023100,86.599317,82.511107,2.637025,15.643696
696,2136,2022.0,-312.033878,26.943856,14.236847,85.965911,82.748450,3.636588,14.791411
697,2137,2022.0,-312.869384,26.542410,14.972372,88.082615,82.626626,3.840101,15.396590
698,2138,2022.0,-308.938303,27.599339,12.882619,84.585012,82.632252,3.306362,14.185139


In [72]:
y_t_pred = regressor_t.predict(df_test_2)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


In [73]:
df_test_3 = df_test_2
df_test_3["T2M"] = y_t_pred
df_test_3

,ID,YEAR,T2M_RANGE,T2M_MAX,T2M_MIN,RH2M,PS,WS10M,QV2M,T2M
0,1440,2009.0,-309.569706,32.919144,17.735611,74.415249,82.443918,6.513046,15.378905,25.362398
1,1441,2009.0,-310.763533,32.340370,17.955298,75.232482,82.320169,5.163099,15.590121,25.368224
2,1442,2009.0,-313.096292,29.670217,17.788919,80.950265,82.456590,3.974139,15.986008,24.017616
3,1443,2009.0,-312.110915,29.586632,16.777071,82.254238,82.460926,5.504794,15.744121,23.144099
4,1444,2009.0,-312.445008,30.088295,17.233512,81.311915,82.326908,5.634695,15.898722,23.692814
...,...,...,...,...,...,...,...,...,...,...
695,2135,2022.0,-310.857597,28.357413,15.023100,86.599317,82.511107,2.637025,15.643696,21.680640
696,2136,2022.0,-312.033878,26.943856,14.236847,85.965911,82.748450,3.636588,14.791411,20.453045
697,2137,2022.0,-312.869384,26.542410,14.972372,88.082615,82.626626,3.840101,15.396590,20.704048
698,2138,2022.0,-308.938303,27.599339,12.882619,84.585012,82.632252,3.306362,14.185139,19.984238


In [74]:
# X_vr_input_scaled = scaler.transform(df_test_3)
# y_vr_pred_res = model.predict(X_vr_input_scaled)

In [75]:
# y_vr_pred_result_gbr = model_vacation_rate.predict(df_test_3)


In [76]:
Y_pred_XGB_result = best_xgb_model.predict(df_test_3)
df_test_4 = df_test_3
df_test_4["VACATION_RATE"] = Y_pred_XGB_result
df_test_4

,ID,YEAR,T2M_RANGE,T2M_MAX,T2M_MIN,RH2M,PS,WS10M,QV2M,T2M,VACATION_RATE
0,1440,2009.0,-309.569706,32.919144,17.735611,74.415249,82.443918,6.513046,15.378905,25.362398,18.545074
1,1441,2009.0,-310.763533,32.340370,17.955298,75.232482,82.320169,5.163099,15.590121,25.368224,18.683319
2,1442,2009.0,-313.096292,29.670217,17.788919,80.950265,82.456590,3.974139,15.986008,24.017616,16.904346
3,1443,2009.0,-312.110915,29.586632,16.777071,82.254238,82.460926,5.504794,15.744121,23.144099,16.094336
4,1444,2009.0,-312.445008,30.088295,17.233512,81.311915,82.326908,5.634695,15.898722,23.692814,15.011729
...,...,...,...,...,...,...,...,...,...,...,...
695,2135,2022.0,-310.857597,28.357413,15.023100,86.599317,82.511107,2.637025,15.643696,21.680640,20.563143
696,2136,2022.0,-312.033878,26.943856,14.236847,85.965911,82.748450,3.636588,14.791411,20.453045,29.698591
697,2137,2022.0,-312.869384,26.542410,14.972372,88.082615,82.626626,3.840101,15.396590,20.704048,23.743061
698,2138,2022.0,-308.938303,27.599339,12.882619,84.585012,82.632252,3.306362,14.185139,19.984238,35.863121


In [77]:
df_test_4.to_csv('solution.csv', columns=['ID', 'VACATION_RATE'], index = False)